In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Inittialize the Spark session
spark = SparkSession.builder.appName('TerraBankETL').getOrCreate()

TypeError: 'JavaPackage' object is not callable

In [ ]:
spark

### Data Extraction

In [ ]:
terra_bank_df = spark.read.csv(r'dataset/terra_bank_transactions.csv', header=True, inferSchema=True)

In [ ]:
# terra_bank_df.printShow()

In [ ]:
terra_bank_df.printSchema()